<a href="https://colab.research.google.com/github/Megha2004/CRM_EmailAutomation/blob/main/Email_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sending mail to specific people**

In [ ]:
import smtplib
import ssl
from email.message import EmailMessage
import os

# Define email sender and receiver
email_sender = 'meghaganesan294@gmail.com'
email_password = 'rxyo rzvk mofv bwoz'
email_receiver = 'lavanyamohan25204@gmail.com'

# Set the subject and body of the email
subject = 'Email with Image Attachment'
body = """
This email contains an attached image.
"""

# Create an EmailMessage
em = EmailMessage()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = subject
em.set_content(body)

# Attach an image to the email
# Replace 'path_to_your_image' with the actual path to the image you want to send
image_filename = '/content/CRM_Deals[1] (2).pbix'  # Provide a desired name for the attached image
with open('/content/CRM_Deals[1] (2).pbix', 'rb') as image_file:
    em.add_attachment(image_file.read(), maintype='image', subtype='pbix', filename=image_filename)

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.send_message(em)


**Sending mail to email in dataset**

In [ ]:
import smtplib
import ssl
from email.message import EmailMessage
import csv

# Define email sender
email_sender = 'meghaganesan294@gmail.com'
email_password = 'rxyo rzvk mofv bwoz'

# Set the subject and body of the email
subject = 'Update from Your CRM System'
body = """
Dear recipient,

This is an automated update from your CRM system.

Best regards,
Your Name
"""

# Create an EmailMessage
em = EmailMessage()
em['From'] = email_sender
em['Subject'] = subject
em.set_content(body)

# Add SSL (layer of security)
context = ssl.create_default_context()

# Load the dataset containing email addresses
# Assuming you have a CSV file with a 'email' column
with open('/content/leads_email.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        email_receiver = row['email']
        specific_people = ['953621243033@ritrjpm.ac.in']

        if email_receiver in specific_people:
            # Log in and send the email
            with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
                smtp.login(email_sender, email_password)
                em['To'] = email_receiver
                smtp.send_message(em)


**Sending mail to clients**

In [ ]:
!python -m spacy download en_core_web_lg

2023-10-30 08:21:45.546991: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 08:21:45.547055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 08:21:45.547092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 08:21:46.787660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import imaplib
import email
import smtplib
import ssl
from email.message import EmailMessage
import csv
import spacy

# Load SpaCy model and dataset
nlp = spacy.load("en_core_web_lg")

# Load your dataset with patterns and responses from a CSV file
dataset = []
with open('/content/chatbot.csv', newline='', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dataset.append({"Pattern": row["Pattern"], "Response": row["Response"]})

# Define email sender
email_sender = 'meghaganesan294@gmail.com'
email_password = 'rxyo rzvk mofv bwoz'

# Add SSL (layer of security)
context = ssl.create_default_context()

# Connect to your email server and fetch incoming emails (IMAP)
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(email_sender, email_password)
mail.select('inbox')

# Search for and fetch unread emails
status, email_ids = mail.search(None, 'UNSEEN')
email_ids = email_ids[0].split()

# Iterate through unread emails
for email_id in email_ids:
    status, email_data = mail.fetch(email_id, '(RFC822)')
    msg = email.message_from_bytes(email_data[0][1])

    # Extract the client's question from the email content
    email_subject = msg['Subject']
    email_payload = msg.get_payload()
    client_question = ""

    if email_payload is not None:
        if isinstance(email_payload, list):
            for part in email_payload:
                if part.get_content_type() == "text/plain":
                    try:
                        client_question = part.get_payload(decode=True).decode('utf-8', 'ignore')
                    except UnicodeDecodeError:
                        client_question = part.get_payload(decode=True, errors='ignore').decode('iso-8859-1', 'ignore')
                    break  # Use the first plain text part
        elif email_payload.get_content_type() == "text/plain":
            try:
                client_question = email_payload.get_payload(decode=True).decode('utf-8', 'ignore')
            except UnicodeDecodeError:
                client_question = email_payload.get_payload(decode=True, errors='ignore').decode('iso-8859-1', 'ignore')

    # Combine email subject and body as the client's question
    client_question = f"{email_subject}\n\n{client_question}"

    # Calculate similarity between client question and dataset patterns
    similarities = {}
    for data in dataset:
        similarity = nlp(client_question).similarity(nlp(data["Pattern"]))
        similarities[data["Pattern"]] = similarity

    # Find the response for the best matching pattern
    best_match = max(similarities, key=similarities.get)
    response = next(item["Response"] for item in dataset if item["Pattern"] == best_match)

    # Send an email response to the client
    email_receiver = msg['From']
    subject = 'Response from CRM'
    body = response  # Set the email body to the response

    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    # Log in and send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.send_message(em)

    # Mark the email as read
    mail.store(email_id, '+FLAGS', '\\Seen')

# Disconnect from the email server
mail.logout()


('BYE', [b'LOGOUT Requested'])

**Email with template**

In [ ]:
import imaplib
import email
import smtplib
import ssl
from email.message import EmailMessage
import csv
import spacy

# Load SpaCy model and dataset
nlp = spacy.load("en_core_web_lg")

# Load your dataset with patterns and responses from a CSV file
dataset = []
with open('/content/chatbot.csv', newline='', encoding='ISO-8859-1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dataset.append({"Pattern": row["Pattern"], "Response": row["Response"]})

# Define email sender
email_sender = 'meghaganesan294@gmail.com'
email_password = 'rxyo rzvk mofv bwoz'

# Load the HTML email template
with open('test.html', 'r') as file:
    template = file.read()

# Add SSL (layer of security)
context = ssl.create_default_context()

# Connect to your email server and fetch incoming emails (IMAP)
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(email_sender, email_password)
mail.select('inbox')

# Search for and fetch unread emails
status, email_ids = mail.search(None, 'UNSEEN')
email_ids = email_ids[0].split()

# Iterate through unread emails
for email_id in email_ids:
    status, email_data = mail.fetch(email_id, '(RFC822)')
    msg = email.message_from_bytes(email_data[0][1])

    # Extract the client's question from the email content
    email_subject = msg['Subject']
    email_payload = msg.get_payload()
    client_question = ""

    if email_payload is not None:
        if isinstance(email_payload, list):
            for part in email_payload:
                if part.get_content_type() == "text/plain":
                    try:
                        client_question = part.get_payload(decode=True).decode('utf-8', 'ignore')
                    except UnicodeDecodeError:
                        client_question = part.get_payload(decode=True, errors='ignore').decode('iso-8859-1', 'ignore')
                    break  # Use the first plain text part
        elif msg.get_content_type() == "text/plain":
            try:
                client_question = email_payload.get_payload(decode=True).decode('utf-8', 'ignore')
            except UnicodeDecodeError:
                client_question = email_payload.get_payload(decode=True, errors='ignore').decode('iso-8859-1', 'ignore')

    # Combine email subject and body as the client's question
    client_question = f"{email_subject}\n\n{client_question}"

    # Calculate similarity between client question and dataset patterns
    similarities = {}
    for data in dataset:
        similarity = nlp(client_question).similarity(nlp(data["Pattern"]))
        similarities[data["Pattern"]] = similarity

    # Find the response for the best matching pattern
    best_match = max(similarities, key=similarities.get)
    response = next(item["Response"] for item in dataset if item["Pattern"] == best_match)

    # Send an email response to the client
    email_receiver = msg['From']
    subject = 'Response from CRM'

    # Extract the client's name from the email sender's address
    email_receiver = msg['From']
    client_name = email_receiver.split('<')[0].strip()

    # After extracting the client's name from the email, replace it in the email template
    template = template.replace('{client_name}', client_name)

    # Replace the {response} placeholder with the actual response
    email_content = template.replace('{response}', response)

    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(email_content, subtype='html') # Set the email body to the response with HTML content

    # Log in and send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.send_message(em)

    # Mark the email as read
    mail.store(email_id, '+FLAGS', '\\Seen')

# Disconnect from the email server
mail.logout()


('BYE', [b'LOGOUT Requested'])